In [1]:
import numpy as np
import torch
import librosa
import librosa.display
import pandas as pd
from IPython.display import Audio
import seaborn as sns
import os

from collections import defaultdict

import tqdm
sns.set()

import matplotlib.pyplot as plt
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.model_selection import train_test_split
import utils

%load_ext autoreload
%autoreload 2

In [2]:
import utils
import models
import trainer

In [3]:
class conf:
    # Preprocessing settings
    sampling_rate = 44100
    duration = 2
    hop_length = 347*duration # to make time steps 128
#     hop_length = 694*duration
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    samples = sampling_rate * duration
    trim_long_data = True

In [4]:
def show(audio):
    return Audio(audio, rate=conf.sampling_rate)

In [5]:
conf.columns, conf.column_encoder = utils.read_columns()

In [6]:
all_data, all_y, max_len = utils.read_dataset("./data/train_curated", conf, lambda s: s)

100%|██████████| 4970/4970 [02:42<00:00,  7.75it/s] 


In [7]:
lengths = np.array(list([len(a) for a in all_data]))/conf.sampling_rate

In [8]:
np.quantile(lengths, 0.95)

2.0

In [9]:
all_data = np.array(all_data)

In [10]:
show(all_data[234] + 10 * all_data[1353] + all_data[1334] + 0.1 * np.random.randn(*all_data[0].shape))

In [11]:
all_data[lengths > 45][0].shape[0]/conf.sampling_rate

IndexError: index 0 is out of bounds for axis 0 with size 0

In [12]:
conf.columns[np.nonzero(all_y[lengths > 45][0])]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [13]:
all_data[0].max()

0.0932312

In [81]:
x, y, coefs, ids = trainer.mix_sounds(all_data, all_y, 3)

In [48]:
show(x)

In [49]:
loader = trainer.SoundAugDataset(all_data, all_y, lambda s: utils.audio_to_melspectrogram(conf, s), 10000, 3, 42)

In [50]:
conf.columns[y > 0]

Index(['Chewing_and_mastication', 'Scissors', 'Screaming'], dtype='object')

In [54]:
conf.columns[all_y[ids][2] > 0]

Index(['Scissors'], dtype='object')

In [82]:
coefs

array([0.67539879, 0.03807556, 0.28652565])

In [78]:
np.random.randint(1, 3)

1